# Assignment 19
The storage system of a small company is composed by two NAS.
Each of them is configured to serve requests in parallel (i.e. processor sharing), and serves two type of users: employees and costumers.

Requests of both type of users arrive according to Poisson processes respectively of rates:
- $\lambda_{e}$ = 0.1 $\frac{req}{s}$
- $\lambda_{c}$ = 10 $\frac{req}{s}$

Each request accesses files from both NASes. The demand of the two NASes for the two types of users have been measured as follows:
- $D_{1E}$ = 2 $s$
- $D_{1C}$ = 0.06 $s$
- $D_{2E}$ = 5 $s$
- $D_{2C}$ = 0.04$s$

## Requests
Determine:
1. Wether the considered system is stable
2. The utilization of the two NAS
3. The residence time of the two NAS
4. The response time of the system

## Characterization

The model I'm going to use takes into consideration **two job classes**: one for the requests from the employees and one for the requests from the customers.

Both classes are open and they are characterized by the arrival rates and the demands of the respective jobs.

In [1]:
import numpy as np

D = np.array([[2, 5],
             [0.06, 0.04]])
l = np.array([0.1, 10])

## Utilization
The utilization of a station for a class can be computed as $$U_{kc} = X_{c} \cdot D_{kc} = \lambda_{c} \cdot D_{kc}$$ 

The utilization of the station (independently of the class) can then be computed as $$U_{k} = \sum_{c}U_{kc}$$

In [2]:
U_k = np.matmul(np.transpose(l), D)
U_kc = np.array([la * D[i] for i,la in enumerate(l)])

print("The utilization for employees on NAS 1 is {}".format(U_kc[0][0]))
print("The utilization for employees on NAS 2 is {}".format(U_kc[0][1]))
print("The utilization for costumers on NAS 1 is {}".format(U_kc[1][0]))
print("The utilization for costumers on NAS 2 is {}".format(U_kc[1][1]))

for i, u in enumerate(U_k):
    print("The utilization of NAS {} is {}".format(i+1, u))

The utilization for employees on NAS 1 is 0.2
The utilization for employees on NAS 2 is 0.5
The utilization for costumers on NAS 1 is 0.6
The utilization for costumers on NAS 2 is 0.4
The utilization of NAS 1 is 0.8
The utilization of NAS 2 is 0.9


## Stability
To be considered stable, every station in the network must have an utilization less than 1.

In [3]:
stability = all(U_k < 1)
print("The system is {}.".format("stable" if stability else "unstable"))

The system is stable.


## Residence times of each station

The residence time of each station can be computed starting from the residence times of each station *for each class* $$R_{kc} = \frac{D_{kc}}{1 - U_{k}}$$

At which point, we can compute hte residence time of each station as $$R_{k} = \sum_{c}\frac{X_{c}}{X}\cdot R_{kc}$$
where $X$ can be computed as the sum of the various $X_{c}$.

In [4]:
R_kc = np.array([
    [d /(1-U_k[0]) for d in D[:,0]],
    [d /(1-U_k[1]) for d in D[:,1]]
])
R_kc = np.transpose(R_kc)

R_k = np.matmul(l/sum(l), R_kc)

for s, st in enumerate(["NAS 1", "NAS 2"]):
    for c, cl in enumerate(["employees", "customers"]):
        print("The residence time for {} in {} is {:.3f}".format(cl, st, R_kc[c][s]))
    print("The response time for {} is {:.3f}".format(st, R_k[s]))


The residence time for employees in NAS 1 is 10.000
The residence time for customers in NAS 1 is 0.300
The response time for NAS 1 is 0.396
The residence time for employees in NAS 2 is 50.000
The residence time for customers in NAS 2 is 0.400
The response time for NAS 2 is 0.891


## Response time of the system
The response time of the system can be computed as $$R = \sum_{k} R_{k}$$

In [5]:
R = np.sum(R_k)

print("The response time of the system is {:3f}".format(R))

The response time of the system is 1.287129
